In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from scipy import ndimage, optimize
from scipy.fft import fft2, ifft2
import cv2
import scipy.constants as c
import pandas as pd
from IPython.display import display

import beam_caustic


# Time of Flight Analysis

In [2]:
def run_d4sigma(
    image: np.ndarray,
    *,
    pixel_size_x_mm: float = 8.4e-3,
    pixel_size_y_mm: float = 9.8e-6,
    self_conf_width: float = 3,
    bkg_ellipse_axes_scaling: float = 1.0,
    d4s_rel_tol: float | None = None,
    d4s_abs_tol: float = 1.0,
    debug_d4sigma: bool = False,
    make_fit_plots: bool = False,
    plot_label: str = "",
):
    """
    Analyze a single CCD sub-image (e.g., before-shutter or after-shutter region).

    Returns
    -------
    fit_dict : dict
        Parameters from the 2D Gaussian fit (from beam_caustic.fit_gaussian_2d).
    gaussian_model : callable
        Callable model gaussian_2d(*popt) that can be evaluated on coordinates.
    d4sigma_summary : dict
        Key results from the D4σ method.
    """

    # ----------------------------
    # 1) Estimate background region using an ellipse fit
    # ----------------------------
    xc_ell, yc_ell, minor_saxis_ell, major_saxis_ell, orientation_ell = beam_caustic.find_ellipse(image)

    ellipse_mask = beam_caustic.get_ellipse_mask(
        xc_ell, yc_ell,
        bkg_ellipse_axes_scaling * minor_saxis_ell,
        bkg_ellipse_axes_scaling * major_saxis_ell,
        orientation_ell,
        image.shape
    )

    # Background estimate from pixels *outside* the ellipse
    if np.any(~ellipse_mask):
        bkg_avg = float(image[~ellipse_mask].mean())
    else:
        # Fallback: if ellipse covers everything, assume zero background
        bkg_avg = 0.0

    image_bkg_corr = image.astype(float) - bkg_avg

    # ----------------------------
    # 2) Fit a 2D Gaussian (for center + waist estimates)
    # ----------------------------
    fit_result, fit_dict, gaussian_2d = beam_caustic.fit_gaussian_2d(
        image_bkg_corr,
        pstart=[
            image_bkg_corr.max(),
            xc_ell, yc_ell,
            minor_saxis_ell, major_saxis_ell,
            orientation_ell,
            0.0
        ],
    )

    # Convert (w0x, w0y) [pixels] -> 1/e^2 diameters in mm (matching your original)
    w_major_mm = 2 * fit_dict["w0x_Value"] * pixel_size_x_mm
    w_minor_mm = 2 * fit_dict["w0y_Value"] * pixel_size_y_mm

    # ----------------------------
    # 3) Run D4σ method (for ISO-style beam widths)
    # ----------------------------
    xc_d4s, yc_d4s, dx_d4s, dy_d4s, orientation_d4s, iterations_d4s, converged_d4s = (
        beam_caustic.run_d4sigma_method(
            image_bkg_corr,
            self_conf_width=self_conf_width,
            debug=debug_d4sigma,
            ignore_crop_error=True,
            rel_tol=d4s_rel_tol,
            abs_tol=d4s_abs_tol,
        )
    )

    d4sigma_summary = {
        "x0_px": float(xc_d4s),
        "y0_px": float(yc_d4s),
        "D4sigma_x_px": float(dx_d4s),
        "D4sigma_y_px": float(dy_d4s),
        "phi_deg": float(orientation_d4s),
        "iterations": int(iterations_d4s),
        "converged": bool(converged_d4s),
        "bkg_avg_cts": float(bkg_avg),
    }

    # ----------------------------
    # 4) Optional: diagnostic plots for this sub-image
    # ----------------------------
    if make_fit_plots:
        fig, axs = plt.subplots(1, 3, figsize=(14, 3.5), constrained_layout=True)
        fig.suptitle(f"2D Gaussian fit diagnostics {plot_label}".strip())

        image_fit = gaussian_2d(*fit_result["Popt"])(*np.flipud(np.indices(image_bkg_corr.shape)))
        images = [
            (image_bkg_corr, "Background-corrected"),
            (image_fit, "Gaussian fit"),
            (image_fit - image_bkg_corr, "Residual (fit - data)"),
        ]

        for ax, (im, title) in zip(axs, images):
            h = ax.imshow(im, cmap="gray")
            ax.set_title(title)
            ax.set_xlabel("x (px)")
            ax.set_ylabel("y (px)")
            plt.colorbar(h, ax=ax, fraction=0.046, pad=0.04)

        # Add a small text box with key numbers
        text = (
            f"x0={fit_dict['x0_Value']:.2f} px, y0={fit_dict['y0_Value']:.2f} px\n"
            f"w_major={w_major_mm:.3e} mm, w_minor={w_minor_mm:.3e} mm\n"
            f"D4σx={dx_d4s:.2f} px, D4σy={dy_d4s:.2f} px, converged={converged_d4s}"
        )
        axs[2].text(1.02, 0.5, text, transform=axs[2].transAxes, va="center")

    # Return raw fit dict + model, plus a compact D4σ summary
    return fit_dict, gaussian_2d, d4sigma_summary


In [3]:
def get_params(filename: str, *, show_raw: bool = False, show_split: bool = True, make_fit_plots: bool = False):
    """
    Load a CCD text file, split into before/after shutter halves, and analyze both regions.

    Returns
    -------
    results : dict
        Contains per-region fit outputs and a small summary table.
    """
    fname = Path(filename).name

    # ---- Load CCD array (your file format: 3 header rows, then pixel values)
    pixel_array = np.loadtxt(filename, dtype=np.uint8, delimiter=None, skiprows=3)

    if show_raw:
        plt.figure(figsize=(6, 4))
        h = plt.imshow(pixel_array, cmap="gray", vmin=0, vmax=255)
        plt.colorbar(h, fraction=0.046, pad=0.04)
        plt.title(f"Raw CCD Image: {fname}")
        plt.xlabel("x (px)")
        plt.ylabel("y (px)")
        plt.show()

    # ---- Split into two halves (matching your original indexing)
    # Image shape seems to be (492, 384). You were taking:
    #   before_shutter: bottom half
    #   after_shutter:  top half
    ny, nx = pixel_array.shape
    mid = ny // 2

    after_shutter = pixel_array[0:mid-1, 0:nx]          # top half
    before_shutter = pixel_array[mid+1:ny-1, 0:nx]      # bottom half

    if show_split:
        fig, axs = plt.subplots(1, 2, figsize=(10, 3.5), constrained_layout=True)
        fig.suptitle(f"CCD halves: {fname}")

        # Keep the same scaling for easy comparison
        vmin, vmax = 0, 255
        h0 = axs[0].imshow(before_shutter, cmap="gray", vmin=vmin, vmax=vmax)
        axs[0].set_title(f"Before Shutter: {fname}")
        axs[0].set_xlabel("x (px)")
        axs[0].set_ylabel("y (px)")

        h1 = axs[1].imshow(after_shutter, cmap="gray", vmin=vmin, vmax=vmax)
        axs[1].set_title(f"After Shutter: {fname}")
        axs[1].set_xlabel("x (px)")
        axs[1].set_ylabel("y (px)")

        # One shared colorbar (matches the common scale)
        cbar = fig.colorbar(h1, ax=axs, fraction=0.046, pad=0.04)
        cbar.set_label("Counts (cts/px)")
        plt.show()

    # ---- Analyze each region
    regions = {
        "before_shutter": before_shutter,
        "after_shutter": after_shutter,
    }

    outputs = {}
    rows = []

    for region_name, region_img in regions.items():
        fit_dict, gaussian_model, d4sigma_summary = run_d4sigma(
            region_img,
            make_fit_plots=make_fit_plots,
            plot_label=f"({region_name}) {fname}",
        )

        # 1/e^2 diameter (mm), matching your earlier calculation
        pixel_size_x_mm = 8.4e-3
        pixel_size_y_mm = 9.8e-6
        waist_x_mm = 2 * fit_dict["w0x_Value"] * pixel_size_x_mm
        waist_y_mm = 2 * fit_dict["w0y_Value"] * pixel_size_y_mm

        outputs[region_name] = {
            "fit_dict": fit_dict,
            "gaussian_model": gaussian_model,
            "d4sigma": d4sigma_summary,
            "waist_x_mm": float(waist_x_mm),
            "waist_y_mm": float(waist_y_mm),
        }

        rows.append({
            "file": fname,
            "region": region_name,
            "x0_px": fit_dict["x0_Value"],
            "y0_px": fit_dict["y0_Value"],
            "waist_x_mm (2*w0x*px)": waist_x_mm,
            "waist_y_mm (2*w0y*py)": waist_y_mm,
            "D4sigma_x_px": d4sigma_summary["D4sigma_x_px"],
            "D4sigma_y_px": d4sigma_summary["D4sigma_y_px"],
            "phi_deg": d4sigma_summary["phi_deg"],
            "bkg_avg_cts": d4sigma_summary["bkg_avg_cts"],
            "d4sigma_converged": d4sigma_summary["converged"],
        })

    summary_df = pd.DataFrame(rows)

    return {
        "file": fname,
        "pixel_array": pixel_array,
        "regions": outputs,
        "summary_df": summary_df,
    }


In [4]:
def analyze_tof(tof_s: float, filename: str, *, make_fit_plots: bool = False, show_split: bool = True):
    """
    Compute TOF velocity and a (simple) temperature estimate from before/after shutter positions.

    Parameters
    ----------
    tof_s : float
        Time-of-flight in seconds.
    filename : str
        CCD file to analyze.
    make_fit_plots : bool
        If True, show Gaussian fit diagnostics per region.
    show_split : bool
        If True, show a side-by-side imshow of before vs after shutter.

    Returns
    -------
    results : dict
        vx, vy, Tx, Ty plus a compact per-file summary table.
    """
    out = get_params(filename, show_raw=False, show_split=show_split, make_fit_plots=make_fit_plots)

    # Pull out fitted centers (convert px -> meters using the same factor you had: *1e-3)
    # NOTE: If you have a more accurate px->meter conversion, swap it in here.

    pixel_size_x_mm = 8.4e-3
    pixel_size_y_mm = 9.8e-6

    before = out["regions"]["before_shutter"]["fit_dict"]
    after  = out["regions"]["after_shutter"]["fit_dict"]



    x_init_m  = before["x0_Value"] * 1e-3
    x_final_m = after["x0_Value"]  * 1e-3
    y_init_m  = before["y0_Value"] * 1e-3
    y_final_m = after["y0_Value"]  * 1e-3

    # waist_x_init = before['w0x_Value'] * 2 * pixel_size_x_mm * 1e-3
    # waist_y_init = before['w0y_Value'] * 2 * pixel_size_y_mm * 1e-3
    # waist_x_final = after["w0x_Value"] * 2 * pixel_size_x_mm * 1e-3
    # waist_y_final = after["w0y_Value"] * 2 * pixel_size_y_mm * 1e-3


    waist_x_init = out["regions"]["before_shutter"]['waist_x_mm'] * 2 * 1e-3
    waist_y_init = out["regions"]["before_shutter"]['waist_y_mm'] * 2 * 1e-3
    waist_x_final = out["regions"]["after_shutter"]['waist_x_mm'] * 2 * 1e-3
    waist_y_final = out["regions"]["after_shutter"]['waist_y_mm'] * 2 * 1e-3
    
    vx = (x_final_m - x_init_m) / tof_s
    vy = (y_final_m - y_init_m) / tof_s

    # Temperature estimate (keeping your original structure; adjust if you have a specific model)
    m_rb85 = 85 * c.atomic_mass
    Tx = m_rb85 * (waist_x_final**2 - waist_x_init**2) / (3 * c.k * tof_s**2)
    Ty = m_rb85 * (waist_y_final**2 - waist_y_init**2) / (3 * c.k * tof_s**2)

    # Nicely formatted per-file printout
    fname = Path(filename).name
    pretty = pd.DataFrame([{
        "file": fname,
        "tof (ms)": tof_s * 1e3,
        "vx (m/s)": vx,
        "vy (m/s)": vy,
        "Tx (uK)": Tx*1e6,
        "Ty (uK)": Ty*1e6,
    }])

    display(pretty.style.format({
        "tof (ms)": "{:.3f}",
        "vx (m/s)": "{:.3e}",
        "vy (m/s)": "{:.3e}",
        "Tx (uK)": "{:.3e}",
        "Ty (uK)": "{:.3e}",
    }))

    return {
        "file": fname,
        "vx": float(vx),
        "vy": float(vy),
        "Tx": float(Tx),
        "Ty": float(Ty),
        "fit_summary": out["summary_df"],
    }


# Error Estimation in CCD Time-of-Flight Analysis

Uncertainties are estimated and propagated in the analysis of CCD images which is used to extract atomic cloud position, velocity, temperature, and atom number. 
The approach separates random (statistical) errors from systematic uncertainties and follows standard error-propagation techniques.

---

## 1. Pixel-Level Noise Model

Each CCD pixel value (in counts) is subject to several noise sources:

* **Photon (shot) noise**, which is Poissonian in the number of detected photoelectrons
* **Read noise** from the CCD electronics
* **Dark current noise**
* **Digitization noise** (typically negligible)

Let $N_e$ be the number of detected electrons and $g$ the camera gain (electrons per count). The pixel variance in counts can be approximated as

$$
\sigma_{\text{px}}^2
\approx
\frac{N_e + \sigma_{\text{read},e}^2 + I_{\text{dark}} t}{g^2}
$$

where $\sigma_{\text{read},e}$ is the read noise (electrons), $I_{\text{dark}}$ is the dark current, and $t$ is the exposure time.

If camera parameters are not precisely known, an empirical estimate of $\sigma_{\text{px}}$ can be obtained from the variance of background regions in the image.

---

## 2. Uncertainty in Fitted Image Parameters

Gaussian fits to the atomic cloud yield parameters such as:

* Cloud center: $x_0, y_0$
* Cloud width (waist): $w_x, w_y$
* Background offset and amplitude

When the fit is performed using weighted least squares, the fit covariance matrix provides estimates of the parameter uncertainties:

$$
\sigma_{p_i} = \sqrt{(\text{Cov})_{ii}}
$$

These uncertainties include the effects of pixel noise and correlations between fit parameters.

---

## 3. Position Uncertainty

Fitted centers are obtained in pixel units and converted to physical units using a pixel-to-length scale $s$ (meters per pixel):

$$
x = x_{0,\text{px}}, s
$$

The uncertainty in position is

$$
\sigma_x^2 = (s,\sigma_{x_0})^2 + (x_{0,\text{px}},\sigma_s)^2
$$

where $\sigma_s$ represents uncertainty in the pixel scale or magnification. This term is often a dominant systematic uncertainty.

---

## 4. Velocity Uncertainty

The velocity is computed from the displacement of the cloud center over a known time of flight $t$:

$$
v_x = \frac{x_f - x_i}{t}
$$

Error propagation gives

$$
\sigma_{v_x}^2 = \frac{\sigma_{x_f}^2 + \sigma_{x_i}^2}{t^2} + \frac{(x_f - x_i)^2}{t^4},\sigma_t^2
$$

where $\sigma_t$ is the uncertainty in the time of flight.

---

## 5. Temperature Uncertainty

The temperature is estimated from the expansion of the cloud width during time of flight:

$$
T_x = \frac{m}{3k} \frac{w_f^2 - w_i^2}{t^2}
$$

where $m$ is the atomic mass and $k$ is Boltzmann’s constant.

The propagated uncertainty is

$$
\begin{aligned}

\sigma_{T_x}^2 =& \left( \frac{2m}{3k} \frac{w_f}{t^2} \right)^2 \sigma_{w_f}^2 + 
\left(\frac{2m}{3k} \frac{w_i}{t^2} \right)^2 \sigma_{w_i}^2
[6pt]
&+ \left( \frac{2m}{3k} \frac{w_f^2 - w_i^2}{t^3} \right)^2 \sigma_t^2

\end{aligned}
$$

Uncertainty in the pixel scale contributes to $\sigma_{w_i}$ and $\sigma_{w_f}$ and should be included explicitly.

---

## 6. Atom Number Uncertainty

### Absorption imaging

For absorption imaging, the atom number is computed from the optical depth:

$$
\text{OD}(x,y) = \ln!\left( \frac{I_0 - I_{\text{dark}}}{I - I_{\text{dark}}} \right)
$$

and

$$
N = \sum_{\text{pixels}} \frac{\text{OD}(x,y),A_{\text{px}}}{\sigma_{\text{eff}}}
$$

Uncertainties arise from:

* Shot noise in $I$ and $I_0$
* Background and dark subtraction
* Pixel area calibration $A_{\text{px}}$
* Effective scattering cross section $\sigma_{\text{eff}}$, including uncertainties in detuning, polarization, and saturation intensity

### Fluorescence imaging

For fluorescence imaging,

$$
N \propto \frac{\sum C}{\eta,R_{\text{sc}},t}
$$

with uncertainties dominated by photon counting noise, collection efficiency $\eta$, and the scattering rate $R_{\text{sc}}$.

---

## 7. Monte-Carlo Error Estimation

A robust method to estimate uncertainties is to propagate noise through the full analysis pipeline:

1. Generate synthetic images by adding realistic noise to the raw CCD data.
2. Re-analyze each synthetic dataset using the same fitting and calculation routines.
3. Compute the standard deviation of the extracted quantities ($x$, $v$, $T$, $N$).

This approach naturally captures parameter correlations and nonlinear effects.

Systematic uncertainties (pixel scale, timing calibration, imaging cross section) are estimated separately by varying these parameters within their known uncertainties.

---

## 8. Final Error Reporting

Final results are reported with separate statistical and systematic uncertainties:

$$
T = T_0 \pm \sigma_{\text{stat}} \pm \sigma_{\text{sys}}
$$

where $\sigma_{\text{stat}}$ is obtained from fitting or Monte-Carlo analysis, and $\sigma_{\text{sys}}$ reflects calibration and model uncertainties.


In [5]:
def robust_sigma(x: np.ndarray) -> float:
    """Robust std estimate via MAD."""
    x = np.asarray(x).ravel()
    med = np.median(x)
    mad = np.median(np.abs(x - med))
    return 1.4826 * mad if mad > 0 else float(np.std(x))

def estimate_bg_sigma(region_img: np.ndarray, q: float = 0.35) -> float:
    """
    Estimate background/read noise level in counts using a low-quantile slice
    (assumes background dominates lower tail).
    """
    arr = region_img.astype(float).ravel()
    thresh = np.quantile(arr, q)
    bg = arr[arr <= thresh]
    if bg.size < 50:
        bg = arr
    return robust_sigma(bg)

def add_noise_simple(img: np.ndarray, sigma_read: float, shot_noise: bool = True, rng=None) -> np.ndarray:
    """
    Simple noise model:
      - additive Gaussian read/background noise with std sigma_read (counts)
      - optional Poisson-like shot noise approximated as Gaussian with std sqrt(signal)
    """
    if rng is None:
        rng = np.random.default_rng()

    imgf = img.astype(float)

    noisy = imgf + rng.normal(0.0, sigma_read, size=imgf.shape)

    if shot_noise:
        # Approximate Poisson with Gaussian; clip negative before sqrt
        sig_shot = np.sqrt(np.clip(imgf, 0, None))
        noisy += rng.normal(0.0, 1.0, size=imgf.shape) * sig_shot

    return noisy


## Atom number (calibration)

The 2D Gaussian fit gives a total camera signal  (in “camera counts”), not atoms by itself.
The most robust way to convert to atom number in this lab is to calibrate the camera integral to PD3:

1. For each image, compute a background-subtracted “Gaussian integral” $S_{\rm cam}$ from the fit.
2. For the same shot/time, compute the PD3 atom number $N_{\rm PD}$ from your voltage analysis.
3. Fit a line $N_{\rm PD} \approx \alpha S_{\rm cam} + \beta$ (often $\beta\approx 0$ if background subtraction is good).
4. Use $N \approx \alpha S_{\rm cam} + \beta$ to get atoms from camera images.

This avoids needing camera QE, gain, exact optics transmission, etc. All of that lives in $\alpha$.


In [6]:
import numpy as np

def gaussian_integral_from_fitdict(fit_dict: dict, *, subtract_offset: bool = True) -> float:
    """
    Returns the integrated Gaussian signal in 'camera units' (counts * pixels^2),
    using the 2D Gaussian model parameters from your fit_dict.

    Assumes model form:
        I(x,y) = Offset + Amplitude * exp( ... )
    with widths w0x, w0y (in pixels).

    Integral of Gaussian part (no offset):
        S = Amplitude * (2*pi*w0x*w0y)
    """
    # Required keys from your fitter output
    A = float(fit_dict["Amplitude_Value"])
    sx = float(fit_dict["w0x_Value"])
    sy = float(fit_dict["w0y_Value"])

    S = A * (2.0 * np.pi * sx * sy)

    # NOTE: subtract_offset doesn't change S here because we're integrating only the Gaussian term.
    # Offset subtraction matters if you instead integrate the *full image* over an ROI.
    return S


In [7]:
def calibrate_camera_to_pd(
    filenames: list[str],
    N_pd: np.ndarray,
    *,
    region: str = "after_shutter",
    assume_intercept_zero: bool = True,
) -> dict:
    """
    Build a calibration mapping from camera Gaussian integral S_cam -> atom number N.

    filenames and N_pd must correspond to the SAME shots/times.
    """
    if len(filenames) != len(N_pd):
        raise ValueError("filenames and N_pd must have the same length")

    S_list = []
    for fn in filenames:
        out = get_params(fn, show_raw=False, show_split=False, make_fit_plots=False)
        fit_dict = out["regions"][region]["fit_dict"]
        S_list.append(gaussian_integral_from_fitdict(fit_dict))

    S = np.asarray(S_list, dtype=float)
    N_pd = np.asarray(N_pd, dtype=float)

    if assume_intercept_zero:
        alpha = float(np.dot(S, N_pd) / np.dot(S, S))
        beta = 0.0
    else:
        alpha, beta = np.polyfit(S, N_pd, deg=1)

    df = pd.DataFrame({"file": [Path(f).name for f in filenames], "S_cam": S, "N_pd": N_pd})
    return {"alpha": alpha, "beta": beta, "points": df, "region": region}

def camera_to_atoms(S_cam: float, alpha: float, beta: float = 0.0) -> float:
    return float(alpha * S_cam + beta)


## Atom Number with Uncertainties

In [8]:
def calibrate_camera_to_pd_with_uncertainty(
    filenames: list[str],
    N_pd: np.ndarray,
    *,
    region: str = "after_shutter",
    assume_intercept_zero: bool = True,
) -> dict:
    """
    Like your calibrate_camera_to_pd, but also returns uncertainty estimates:
      - alpha, beta
      - alpha_std, beta_std
      - cov (2x2) for [alpha, beta]
    """
    if len(filenames) != len(N_pd):
        raise ValueError("filenames and N_pd must have the same length")

    S_list = []
    for fn in filenames:
        out = get_params(fn, show_raw=False, show_split=False, make_fit_plots=False)
        fit_dict = out["regions"][region]["fit_dict"]
        S_list.append(gaussian_integral_from_fitdict(fit_dict))

    S = np.asarray(S_list, dtype=float)
    y = np.asarray(N_pd, dtype=float)
    n = len(S)

    df = pd.DataFrame({"file": [Path(f).name for f in filenames], "S_cam": S, "N_pd": y})

    if assume_intercept_zero:
        # y = alpha*S, beta = 0
        denom = np.dot(S, S)
        alpha = float(np.dot(S, y) / denom)
        beta = 0.0

        resid = y - alpha * S
        dof = max(n - 1, 1)
        s2 = float(np.dot(resid, resid) / dof)  # residual variance estimate

        alpha_var = s2 / denom
        alpha_std = float(np.sqrt(alpha_var))
        beta_std = 0.0
        cov = np.array([[alpha_var, 0.0],
                        [0.0,      0.0]], dtype=float)

    else:
        # y = alpha*S + beta
        X = np.column_stack([S, np.ones_like(S)])
        # OLS: (X^T X)^-1 X^T y
        XtX = X.T @ X
        XtX_inv = np.linalg.inv(XtX)
        theta = XtX_inv @ (X.T @ y)
        alpha, beta = float(theta[0]), float(theta[1])

        resid = y - (alpha * S + beta)
        dof = max(n - 2, 1)
        s2 = float((resid @ resid) / dof)

        cov = s2 * XtX_inv
        alpha_std = float(np.sqrt(cov[0, 0]))
        beta_std  = float(np.sqrt(cov[1, 1]))

    return {
        "alpha": alpha,
        "beta": beta,
        "alpha_std": alpha_std,
        "beta_std": beta_std,
        "cov": cov,
        "points": df,
        "region": region,
        "assume_intercept_zero": assume_intercept_zero,
    }


# Main Function

In [9]:
def analyze_tof_with_uncertainty(
    tof_s: float,
    filename: str,
    *,
    n_mc: int = 200,
    tof_sigma_s: float = 0.0,                  # timing uncertainty (seconds)
    px_to_mx: float = 1e-3,                    # meters per pixel in x
    px_to_my: float = 1e-3,                    # meters per pixel in y
    px_to_mx_sigma_frac: float = 0.0,          # fractional uncertainty on px_to_mx
    px_to_my_sigma_frac: float = 0.0,          # fractional uncertainty on px_to_my
    shot_noise: bool = True,
    read_noise_override: float | None = None,  # if you know read noise (counts), set it
    include_atoms: bool = False,
    calib: dict | None = None,                 # output of calibrate_camera_to_pd_with_uncertainty
    atom_region: str = "after_shutter",
    rng_seed: int | None = None,
    show_split: bool = False,
):
    """
    Monte-Carlo uncertainty estimate for TOF pipeline with anisotropic pixel scales.

    Uses:
      x [m] = x0_px * px_to_mx
      y [m] = y0_px * px_to_my
      wx [m] = 2*w0x_px*px_to_mx
      wy [m] = 2*w0y_px*px_to_my

    Returns a dict with summary stats + raw MC samples.
    """
    rng = np.random.default_rng(rng_seed)

    # ---- Load once (same split as your get_params) ----
    pixel_array = np.loadtxt(filename, dtype=np.uint8, delimiter=None, skiprows=3)
    ny, nx = pixel_array.shape
    mid = ny // 2
    after_shutter  = pixel_array[0:mid-1, 0:nx].astype(float)
    before_shutter = pixel_array[mid+1:ny-1, 0:nx].astype(float)

    # Estimate read/background noise per region (or override)
    if read_noise_override is None:
        sig_before = estimate_bg_sigma(before_shutter)
        sig_after  = estimate_bg_sigma(after_shutter)
    else:
        sig_before = float(read_noise_override)
        sig_after  = float(read_noise_override)

    # Optional: show split once (no MC)
    if show_split:
        import matplotlib.pyplot as plt
        fig, axs = plt.subplots(1, 2, figsize=(10, 3.5), constrained_layout=True)
        fig.suptitle(f"CCD halves: {Path(filename).name}")
        axs[0].imshow(before_shutter, cmap="gray", vmin=0, vmax=255)
        axs[0].set_title("Before shutter")
        axs[1].imshow(after_shutter, cmap="gray", vmin=0, vmax=255)
        axs[1].set_title("After shutter")
        plt.show()

    # ---- Constants for temperature formula ----
    m_rb85 = 85 * c.atomic_mass
    A = m_rb85 / (3 * c.k)

    # ---- Storage ----
    samples = {
        "vx": [], "vy": [], "Tx": [], "Ty": [],
        "x_init": [], "x_final": [], "y_init": [], "y_final": [],
        "wx_init": [], "wx_final": [], "wy_init": [], "wy_final": [],
        "px_to_mx_used": [], "px_to_my_used": [], "tof_used": [],
    }
    if include_atoms:
        if calib is None:
            raise ValueError("include_atoms=True requires calib=dict from calibrate_camera_to_pd_with_uncertainty.")
        samples["S_cam"] = []
        samples["N_atoms"] = []
        samples["alpha_used"] = []
        samples["beta_used"] = []

    # ---- MC loop ----
    for _ in range(n_mc):
        # Sample TOF and anisotropic pixel scales
        tof_i = float(rng.normal(tof_s, tof_sigma_s)) if tof_sigma_s > 0 else float(tof_s)

        px_to_mx_i = float(rng.normal(px_to_mx, px_to_mx * px_to_mx_sigma_frac)) if px_to_mx_sigma_frac > 0 else float(px_to_mx)
        px_to_my_i = float(rng.normal(px_to_my, px_to_my * px_to_my_sigma_frac)) if px_to_my_sigma_frac > 0 else float(px_to_my)

        # Noisy regions
        before_noisy = add_noise_simple(before_shutter, sigma_read=sig_before, shot_noise=shot_noise, rng=rng)
        after_noisy  = add_noise_simple(after_shutter,  sigma_read=sig_after,  shot_noise=shot_noise, rng=rng)

        # Fit noisy images (your existing fitter)
        fit_b, _, _ = run_d4sigma(before_noisy, make_fit_plots=False, plot_label="")
        fit_a, _, _ = run_d4sigma(after_noisy,  make_fit_plots=False, plot_label="")

        # Centers in meters (anisotropic scaling)
        x_init_m  = float(fit_b["x0_Value"]) * px_to_mx_i
        x_final_m = float(fit_a["x0_Value"]) * px_to_mx_i
        y_init_m  = float(fit_b["y0_Value"]) * px_to_my_i
        y_final_m = float(fit_a["y0_Value"]) * px_to_my_i

        # Waists in meters (anisotropic scaling)
        wx_init  = 2.0 * float(fit_b["w0x_Value"]) * px_to_mx_i
        wx_final = 2.0 * float(fit_a["w0x_Value"]) * px_to_mx_i
        wy_init  = 2.0 * float(fit_b["w0y_Value"]) * px_to_my_i
        wy_final = 2.0 * float(fit_a["w0y_Value"]) * px_to_my_i

        # Derived quantities
        vx = (x_final_m - x_init_m) / tof_i
        vy = (y_final_m - y_init_m) / tof_i
        Tx = A * (wx_final**2 - wx_init**2) / (tof_i**2)
        Ty = A * (wy_final**2 - wy_init**2) / (tof_i**2)

        # Save
        samples["vx"].append(vx); samples["vy"].append(vy)
        samples["Tx"].append(Tx); samples["Ty"].append(Ty)
        samples["x_init"].append(x_init_m); samples["x_final"].append(x_final_m)
        samples["y_init"].append(y_init_m); samples["y_final"].append(y_final_m)
        samples["wx_init"].append(wx_init); samples["wx_final"].append(wx_final)
        samples["wy_init"].append(wy_init); samples["wy_final"].append(wy_final)
        samples["px_to_mx_used"].append(px_to_mx_i)
        samples["px_to_my_used"].append(px_to_my_i)
        samples["tof_used"].append(tof_i)

        # Atom number (optional)
        if include_atoms:
            fit_for_atoms = fit_a if atom_region == "after_shutter" else fit_b
            S_cam = float(gaussian_integral_from_fitdict(fit_for_atoms))
            samples["S_cam"].append(S_cam)

            cov = np.asarray(calib["cov"], dtype=float)
            mu = np.array([calib["alpha"], calib["beta"]], dtype=float)
            alpha_i, beta_i = rng.multivariate_normal(mu, cov)
            samples["alpha_used"].append(alpha_i)
            samples["beta_used"].append(beta_i)

            N_atoms = float(camera_to_atoms(S_cam, alpha_i, beta_i))
            samples["N_atoms"].append(N_atoms)

    # Convert to arrays
    for k in list(samples.keys()):
        samples[k] = np.asarray(samples[k], dtype=float)

    def summarize(arr):
        arr = np.asarray(arr, dtype=float)
        return {
            "mean": float(np.mean(arr)),
            "std":  float(np.std(arr, ddof=1)) if arr.size > 1 else 0.0,
            "p16":  float(np.percentile(arr, 16)),
            "p50":  float(np.percentile(arr, 50)),
            "p84":  float(np.percentile(arr, 84)),
        }

    summary = {
        "file": Path(filename).name,
        "tof_s": float(tof_s),
        "n_mc": int(n_mc),
        "noise_sig_before_counts": float(sig_before),
        "noise_sig_after_counts": float(sig_after),
        "px_to_mx": float(px_to_mx),
        "px_to_my": float(px_to_my),
        "px_to_mx_sigma_frac": float(px_to_mx_sigma_frac),
        "px_to_my_sigma_frac": float(px_to_my_sigma_frac),
        "tof_sigma_s": float(tof_sigma_s),
        "vx": summarize(samples["vx"]),
        "vy": summarize(samples["vy"]),
        "Tx": summarize(samples["Tx"]),
        "Ty": summarize(samples["Ty"]),
    }
    if include_atoms:
        summary["N_atoms"] = summarize(samples["N_atoms"])

    return {"summary": summary, "samples": samples}


# Test Run

In [10]:
# Default Values

pixel_size_x_mm = 8.4e-3
pixel_size_y_mm = 9.8e-3
self_conf_width = 3
bkg_ellipse_axes_scaling = 1.0
d4s_rel_tol = None
d4s_abs_tol = 1.0
debug_d4sigma = False
make_fit_plots = False

In [11]:
res = analyze_tof_with_uncertainty(
    tof_s=5e-3,
    filename="drop4photo1.txt",
    n_mc=400,
    px_to_mx=8.4e-6/0.5,     # example: pixel pitch / magnification
    px_to_my=9.8e-6/0.5,
    px_to_mx_sigma_frac=0.02,
    px_to_my_sigma_frac=0.02,
    tof_sigma_s=2e-6,
    rng_seed=0,
)
res["summary"]["vx"], res["summary"]["Tx"]


Error: Unable to determine beam parameters using moments, stopping


/Users/nadiathesun/Downloads/adinadia12826 MOT data/Physics111B/ccd analysis/beam_caustic.py:170: RuntimeWarning: invalid value encountered in sqrt
  dx = 2*np.sqrt(2)*np.sqrt(xx+yy+(xx-yy)*np.sqrt(1+np.tan(2*orientation)**2))
/Users/nadiathesun/Downloads/adinadia12826 MOT data/Physics111B/ccd analysis/beam_caustic.py:171: RuntimeWarning: invalid value encountered in sqrt
  dy = 2*np.sqrt(2)*np.sqrt(xx+yy-(xx-yy)*np.sqrt(1+np.tan(2*orientation)**2))


2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters 

/Users/nadiathesun/Downloads/adinadia12826 MOT data/Physics111B/ccd analysis/beam_caustic.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  xc = np.einsum('i,ji->', X, image)/total
/Users/nadiathesun/Downloads/adinadia12826 MOT data/Physics111B/ccd analysis/beam_caustic.py:144: RuntimeWarning: divide by zero encountered in scalar divide
  yc = np.einsum('i,ij->', Y, image)/total


Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine beam parameters using moments, stopping
2D Gaussian fit did not converge: Input must be 1- or 2-d.
Error: Unable to determine beam parameters using moments, stopping
Error: Unable to determine 

KeyboardInterrupt: 

# Hard Calculate Atom Number

1. **Counts → photoelectrons**
   $$
   N_{e^-} = \frac{S_{\text{cam}}}{g}
   $$
   where $g$ = camera gain in **counts per electron** (or inverse, depending on convention).

2. **Photoelectrons → detected photons**
   $$
   N_{\gamma,\text{det}} = \frac{N_{e^-}}{\mathrm{QE}}
   $$
   (QE = quantum efficiency at 780 nm-ish).

3. **Detected photons → total scattered photons**
   Only a fraction of emitted photons reach your camera:
   $$
   N_{\gamma,\text{scat}} = \frac{N_{\gamma,\text{det}}}{\eta}
   $$
   where $\eta$ lumps:

* solid angle: $\Omega/4\pi \approx \frac{\pi (D/2)^2}{4\pi L^2} = \frac{D^2}{16L^2}$ (small-angle approx)
* lens transmission
* filter transmission
* camera window
* any losses

4. **Scattered photons → atoms**
   If your exposure time is $t_{\text{exp}}$ and each atom scatters at rate $R_{\text{sc}}$ (photons/s/atom), then
   $$
   N_{\gamma,\text{scat}} \approx N , R_{\text{sc}}, t_{\text{exp}}
   $$
   so
   $$
   \boxed{N \approx \frac{N_{\gamma,\text{scat}}}{R_{\text{sc}},t_{\text{exp}}}}
   $$

And the scattering rate (for a two-level-ish model) is
$$
R_{\text{sc}}=\frac{\Gamma}{2}\frac{s}{1+s+(2\Delta/\Gamma)^2}
$$
with $s=I/I_{\text{sat}}$, detuning $\Delta$, linewidth $\Gamma$.


In [ ]:
# Constants/Measured Inputs 
DIST_MM = 14.5      # distance from MOT to focusing lens [mm]
DIAM_MM = 5.5       # diameter of focusing lens [mm]
DETUNING_MHZ = 25   # detuning (MHz)
GAMMA_RAD_S = 2*np.pi*6.07e6   # Rb D2 natural linewidth (rad/s)
I_SAT = 17.0  # saturation intensity W/m^2  (1.7 mW/cm^2)
OMEGA_MEAN = np.mean([0.008/2, 0.009/2, 0.0092/2, 0.0085/2])  # mean beam radius at MOT (m)
P_X = 0.005426 # power of input beam in X direction (W)
IMPEDANCE_OHM = 1e6 # DAQ analog I/O input impedance (Ohms)
RESPONSIVITY_AW = 0.45 # responsivity of PD3 (A/W)
LAMBDA = 780e-9



# indexes
V_ROOM_BOUND = 0.06
V_ZOOMED = 1600
SWITCH_TIME_DELAY = 25
LOADING_WINDOW_UP_BOUND = None

NameError: name 'fit_dict' is not defined